In [98]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import inflect

p = inflect.engine()

import re

In [99]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
CREDENTIALS_DIR = '.credentials'
CREDENTIALS_PATH = os.path.join(CREDENTIALS_DIR, 'credentials.json')
TOKEN_PATH = os.path.join(CREDENTIALS_DIR, 'token.json')

creds = None
# Creare la directory delle credenziali se non esiste
if not os.path.exists(CREDENTIALS_DIR):
    os.makedirs(CREDENTIALS_DIR)

# Il file token.json memorizza i token di accesso e aggiornamento dell'utente.
if os.path.exists(TOKEN_PATH):
    creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

# Se non ci sono credenziali valide, chiedi all'utente di accedere.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CREDENTIALS_PATH, SCOPES)
        creds = flow.run_local_server(port=0)
    # Salva le credenziali per la prossima esecuzione
    with open(TOKEN_PATH, 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials=creds)
    xai_id = '8881f4je38oo4rbj8qs2cr05lo@group.calendar.google.com'
    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    xai_bday = '2019-01-01T11:30:00+02:00'
    events_result = service.events().list(calendarId=xai_id, timeMin=xai_bday,
                                          maxResults=200, singleEvents=True,
                                          orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')

    # Prints the start and name of the next 10 events
    seminars = []
    for i, event in enumerate(events):
        seminar = {}
        id = event['id']
        imported_event = service.events().get(calendarId=xai_id, eventId=id).execute()

        if '[XAI][SM]' in imported_event['summary']:
            start = imported_event['start'].get('dateTime', imported_event['start'].get('date'))
            seminar['date'] = {}
            seminar['date']['day'] = start.split('T')[0].split('-')[2]
            seminar['date']['month'] = start.split('T')[0].split('-')[1]
            seminar['date']['year'] = start.split('T')[0].split('-')[0]
            seminar['date']['hour'] = start.split('T')[1]
            seminar['complete_date'] = start
            seminar['summary'] = imported_event['summary'].replace('[XAI][SM]', '').strip()
            if 'location' in imported_event:
                seminar['location'] = imported_event['location']
            if 'description' in imported_event:
                seminar['description'] = imported_event['description']

            seminars.append(seminar)

            # if i == 7:
            #     break

except HttpError as error:
    print('An error occurred: %s' % error)

In [100]:
event = seminars[-3]
event

{'date': {'day': '15',
  'month': '02',
  'year': '2024',
  'hour': '11:30:00+01:00'},
 'complete_date': '2024-02-15T11:30:00+01:00',
 'summary': 'GenTree: A Generative Model for Decision Trees',
 'description': '## Presenter. Mattia Setzu ##<br><br>Abstract. Decision trees are among the most popular supervised models due to their interpretability and knowledge representation resembling human reasoning. Commonly-used decision tree induction algorithms are based on greedy top-down strategies. Although these approaches are known to be an efficient heuristic, the resulting trees are only locally optimal and tend to have overly complex structures. \xa0On the other hand, optimal decision tree algorithms attempt to create an entire decision tree at once to achieve global optimality.<br><br>We place our proposal between these approaches by designing a generative model for decision trees. Our method first learns a latent decision tree space through a variational architecture using pre-trained 

In [101]:
def title_to_md(event):
    def remove_special_characters(text):
        return re.sub(r'[^A-Za-z0-9\s]', '', text)

    title = event['summary']
    ltitle = title.split(' ')
    pltitle = [remove_special_characters(x.strip()) for x in ltitle]
    pltitle = [x for x in pltitle if x]
    name = '-'.join(pltitle[:5])
    mdName = event['date']['year'] + '-' + event['date']['month'] + '-' + event['date']['day'] + '-' + name + '.md'
    return mdName


title_to_md(event)

'2024-02-15-GenTree-A-Generative-Model-for.md'

In [102]:
post_list = []
for event in seminars:
    [title_to_md(event) for event in seminars]

In [113]:
# Lista di nomi di file di esempio
# post_list = ['2021-06-01-test.md', '2021-06-02-test-2.md']

# Funzione per rimuovere i caratteri speciali da un testo (ad eccezione di spazi, lettere e numeri e i segni di punteggiatura come virgole punti due punti e trattini)

def special_remove(text):
    return re.sub(r'[^A-Za-z0-9\s.,:;-<>#]', '', text)



def create_file_in_post_folder(event):
    # Event variable is a dictionary with keys: 'summary', 'complete_date', 'description'
    title = special_remove(event['summary'])
    date = event['complete_date'].replace('T', ' ').split('+')[0]
    year = date.split('-')[0]
    if 'location' in event:
        location = event['location']
    else:
        location = ''
    description = special_remove(event['description'])
    
    if '##' in description:
        presenter = description.split('##')[1].strip()
        # Remove html element from presenter variable string:
        presenter = re.sub('<[^<]+?>', '', presenter)
        description = description.split('##')[2]

    # Assicurati che la directory _posts/xai esista
    directory = "_posts/xai"
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = title_to_md(event)
    file_path = os.path.join(directory, filename)

    # Controlla se il file esiste già, altrimenti crealo
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            # Puoi aggiungere contenuto di default se necessario
            f.write(f'''---
layout: post
title: "{title}"
date: "{date}"
year: {year}
tags: "seminar"
location: "{location}"
presenter: "{presenter}"
---
<h5>{presenter}</h5>
<em>Location: {location}<em>
<br>
<hr>
{description}
                ''')  # File vuoto
        print(f"File created: {file_path}")
    else:
        print(f"File already exists: {file_path}")


# Esegui la funzione per ogni nome evento
for event in seminars:
    create_file_in_post_folder(event)
# Chiama la funzione con la lista di nomi di file


In [108]:
seminars[-3]['complete_date']

'2024-02-15T11:30:00+01:00'

In [105]:
#reformat string like this 2024-01-27 00:20:00
def reformat_date(date):
    return date.replace('T', ' ').split('+')[0]


reformat_date(seminars[0]['complete_date'])

'2022-06-09 17:00:00'

In [106]:
# escape double quote characters in a string
def escape_double_quotes(text):
    return text.replace('"', '\\"')